In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
def add_conv_layers(model, input_shape, conv_filters, kernel_size, activation, pool_size, dropout_rates):
    model.add(Input(shape=input_shape))
    for i, filters in enumerate(conv_filters):
        model.add(Conv1D(filters, kernel_size, activation=activation))
        model.add(BatchNormalization())
        model.add(MaxPooling1D(pool_size))
        model.add(Dropout(dropout_rates[i]))

In [3]:
def add_dense_layers(model, dense_units, activation, dropout_rate, num_classes, l2_reg):
    model.add(Flatten())
    model.add(Dense(dense_units, activation=activation, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

In [4]:
def compile_model(model, learning_rate):
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

In [5]:
def create_model(input_shape, num_classes, conv_filters=[32, 64, 128], 
                 kernel_size=(3, 3), activation='relu', 
                 pool_size=(2, 2), dropout_rates=[0.25, 0.25, 0.25, 0.5], 
                 dense_units=256, learning_rate=0.00001, l2_reg=0.01):
    
    model = Sequential()
    
    add_conv_layers(model, input_shape, conv_filters, kernel_size, activation, pool_size, dropout_rates)
    add_dense_layers(model, dense_units, activation, dropout_rates[-1], num_classes, l2_reg)
    compile_model(model, learning_rate)
    
    return model

In [6]:
import pandas as pd
def prepare_data(train_dir, val_dir):
    # Đọc dữ liệu từ tệp CSV
    train_data = pd.read_csv(train_dir)
    val_data = pd.read_csv(val_dir)

    # Tạo generator cho tập huấn luyện
    train_data = train_data.drop(columns=['file_path'])
    val_data = val_data.drop(columns=['file_path'])

    # Tạo generator cho tập validation
    train_generator = (train_data.iloc[:, :-1].values, train_data.iloc[:, -1].values)
    validation_generator = (val_data.iloc[:, :-1].values, val_data.iloc[:, -1].values)

    return train_generator, validation_generator

In [7]:
def train_model(model, train_generator, validation_generator, epochs=50, patience=8):
    steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = validation_generator.samples // validation_generator.batch_size
    
    # Sử dụng EarlyStopping để ngừng huấn luyện sớm nếu không cải thiện
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)

    # Huấn luyện mô hình
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        epochs=epochs,
        callbacks=[early_stopping]
    )

    return history

In [8]:
def plot(history):
  # Vẽ biểu đồ mất mát
  plt.figure(figsize=(12, 4))
  plt.subplot(1, 2, 1)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title('Loss over epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  # Vẽ biểu đồ độ chính xác
  plt.subplot(1, 2, 2)
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.title('Accuracy over epochs')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  plt.show()

In [9]:
# Chuẩn bị dữ liệu
train_dir = "D:\data_analysis\speech_emotion_recognition\src\feature_extraction\merged_feature_data.csv"
val_dir = "D:\data_analysis\speech_emotion_recognition\src\feature_extraction\val_feature_data.csv"
train_generator, validation_generator = prepare_data(train_dir, val_dir)

# Xác định số lớp và hình dạng đầu vào
num_classes = len(train_generator.class_indices)
input_shape = (128, 128, 3)

# Tạo và huấn luyện mô hình
model = create_model(input_shape, num_classes, conv_filters=[32, 64, 128], 
                    kernel_size=(3, 3), activation='relu', 
                    pool_size=(2, 2), dropout_rates=[0.25, 0.25, 0.25, 0.5], 
                    dense_units=256, learning_rate=0.0001, l2_reg=0.01)
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size
history = train_model(model, train_generator, validation_generator, steps_per_epoch, validation_steps)

# Vẽ biểu đồ và lưu mô hình
plot(history)
model.save('emotion_recognition_model.h5')

OSError: [Errno 22] Invalid argument: 'D:\\data_analysis\\speech_emotion_recognition\\src\x0ceature_extraction\\merged_feature_data.csv'